# Joint Intent detection and slot filling
Dieses Jupyter notebook wurde als semesterabschließende Arbeit für das Modul Natural Language Processing an der [Fachhochschule Südwestfalen](https://www.fh-swf.de/en/international_3/index.php) erstellt.

## Einleitung
Das Joint intent detection and slot filling (IDSF) ist eine Aufgabe aus dem Teilbereich des Natural Language Understandings (NLU) des Natural Language Processings (NLP), die uns in heutzutage fast täglich Alltag begegnet. Sei es um einen Timer auf dem Handy zu starten, bestimmte Musik abzuspielen oder das Licht einzuschalten. Der Ablauf ist dabei häufig der selbe: "Siri stelle einen Timer für 4 Minuten", "Alexa spiele meine Schlager Playlist" oder "Google erstelle einen Arzttermin für heute 16:00 Uhr". Meist beginnen die Kommandos mit dem Namen des Sprachassistenten, um diesen zu aktivieren, gefolgt vom Kommando für die gewünschte Aktion. Das IDSF beschäftigt sich dabei mit der Aufgabe, die gewünschte Aktion (Intent), also stelle einen Timer, Spiele Musik, erstelle einen Termin im Kalender und die dazugehörigen notwendigen Parameter, wie z.B. vier Minuten, Schlager Playlist oder Arzt heute 16:00 Uhr (Slots) zu erkennen.
Da der Gebrauch dieser Sprachassistenten in Zukunft wahrscheinlich noch stärker zu nehmen wird, wollen wir uns deren funktionsweise in diesem Notebook näher anschauen. Dafür wird zuerst die Entwicklungshistorie vom IDSF betrachtet und anschließend wird ein eigenes Modell für die Erkennung erstellt und anhand eines selbst vorbereiteten Korpus trainiert.

## Joint Intent Detection and Slot filling

--------------

## Datenbeschaffung

Als Datensatz für das nachfolgende Beispiel verwenden wir den Snips-Datensatz. Dieser Datensatz wurde vom, mittlerweil zu Sonos gehörenden [1], [Snips Team](https://snips.ai/) zusammengestellt, um ihr eigenes Modell mit anderen Wettbewerbern wie zum Beispiel Amazons Alexa zu verglichen. Die Ergebnisse und die Datensätze der drei Vergleiche wurden in einem [GitHub Repository](https://github.com/sonos/nlu-benchmark/tree/master) veröffentlicht und in dem Paper "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces" [2] erläutert. Das Repository enthält die Daten für drei Evaluationen aus den Jahren 2016 bis 2018. Wir werden in diesem Notebook die Daten der 2017 durchgeführten Evaluation verwenden, da diese Sätze für sieben unterschiedliche und allgemeine Aufgaben enthält.

Die Daten sind im dem Repository in einzelnen JSON-Dateien enthalten. Dabei gibt es pro Aufgabe zwei Dateien, eine für das Training und eine für die Validierung. Der Einfachheit halber wurden die Dateien in dem data Verzeichnis, dass diesem Notebook beiliegt, abgelegt.

Nachfolgend ist ein Auszug aus der `train_AddToPlaylist_full.json`-Datei.

In [15]:
!head -n 48 data/train_AddToPlaylist_full.json # Zeige die ersten 23 Zeilen der angegebenen Datei an.

{
  "AddToPlaylist": [
    {
      "data": [
        {
          "text": "Add another "
        },
        {
          "text": "song",
          "entity": "music_item"
        },
        {
          "text": " to the "
        },
        {
          "text": "Cita Romántica",
          "entity": "playlist"
        },
        {
          "text": " playlist. "
        }
      ]
    },
    {
      "data": [
        {
          "text": "add "
        },
        {
          "text": "clem burke",
          "entity": "artist"
        },
        {
          "text": " in "
        },
        {
          "text": "my",
          "entity": "playlist_owner"
        },
        {
          "text": " playlist "
        },
        {
          "text": "Pre-Party R&B Jams",
          "entity": "playlist"
        }
      ]
    },


Die Datei besteht an oberster Stelle aus dem Namen der gewünschten Aktion gefolgt von einer Liste an Objekten mit einem `Data` Attribut. Dieses enthält wiederum eine Liste von Objekten mit `Text` Attributen die den Satz in Teilen enthält. Dabei wird der Satz durch den Text eines definierten `Entities` geteilt. So enthält das erste Beispiel den Text bis zum ersten `entity` das als `music_item` klassifiziert wurde und wieder den gesamten Text bis zum nächsten entity, dem Namen einer Playlist.

Als nächstes werden die Daten in ein verwendbares Format transformiert. Ein in der Natural language processing gängiges Format ist das IOB Format. IOB steht für Inside-Outside-Beginning. Dieses Format ermöglicht die Kennzeichnung der einzelnen Entitäten in einem Satz. Es wird unteranderem von den weit verbreiteten Python Bibliotheken `NLTK` und `spaCy` unterstützt [3, 4]. Das Format wurde 1995 von Lance A. Ramshaw und Mitchell P. Marcus erfunden.

Dieses Beispiel zeigt das Format einer Zeile, welches nachfolgend aus den JSON-Dateien erzeugt wird.

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS o o b-mucic_item i-music_item o i-playlist_owner o b-playlist i-playlist i-playlist
    
Am Beginn der Zeile steht der vollständige Satz abgetrennt durch ein BOS (begin of sentence) am Anfang des Satzes und ein EOS (end of sentence) am Ende des Satzes. Nun folgt das eigentliche IOB-Format. Dabei wird für jeden Token entweder der Buchstabe 'o', dieser steht für keine Bedeutung, der Buchstabe 'b', für den Beginn einer Entität die aus mehreren Token besteht, oder 'i', als Entität. Das 'i' steht dabei entweder nach einem 'b' wodurch eine Entität gekennzeichent wird, die aus mehreren Token besteht oder alleine für eine Entität die aus nur einem Token besteht.  Das 'b' und 'i' werden dabei jeweils gefolgt vom einem trennenden Bindestrich und der Entitätskategorie verwender. So ist der Name 'Clem Burke' unterteilt in ein `b-music_item` für Clem und `i-Music_item` für Burke. Dadurch wird definiert, dass die beiden Teile zusammen gehören.

Das IOB2 Format ist eine Erweiterung des originalen IOB Formats. Es definiert das auch eine Entität die nur aus einem Token besteht mit einem 'b' kodiert wird und nicht wie im IOB Format mit einem 'i'. Dadurch ergibt sich das folgende Format:

    BOS add clem burke in my playlist Pre-Party R&B Jams EOS o o b-mucic_item i-music_item o b-playlist_owner o b-playlist i-playlist i-playlist
    

Mit dem folgenden Python Code wird der Inhalt der im data-Verzeichnis liegenden Dateien in das vorgestellte Format transformiert. Die Dateien werden dabei im Verzeichnis `data/corpus` und einem Verzeichnis mit dem Titel der Intent-Kategorie abgelegt. Als Dateinamen werden `UUID`s verwendet. 


In [1]:
import os
import json
import re
from shutil import rmtree
import uuid

In [4]:
def clean_formatted(formatted_directory_path):
    if not os.path.exists(formatted_directory_path) or not os.path.isdir(formatted_directory_path):
        print(f'Pfad {formatted_directory_path} ist kein Verzeichnis oder existiert nicht')
        return

    rmtree(formatted_directory_path)
    print('Alte Corpus-Dateien gelöscht')

def convert_file(json_file_path):
    print(f'Try converting file at path {json_file_path}')
    if not os.path.isfile(json_file_path):
        print(f'File {file_path} does not exists', json_file_path)
        return 
    formatted_lines = []
    intent_category = None
    with open(json_file_path, 'r', encoding='latin-1') as json_file:
        json_content = json.load(json_file)
        intent_category = next(iter(json_content))
        for sentence_block in json_content[intent_category]:
            sentence = ""
            slots = []
            for sentence_data_block in sentence_block['data']:
                sentence_part = sentence_data_block['text']
                sentence_part = re.sub('\n', '', sentence_part)
                if sentence_part != '':
                    sentence += sentence_part
                sentence_part_len = len(sentence_part.split())
                if 'entity' in sentence_data_block:
                    entity_type = sentence_data_block['entity']
                    if sentence_part_len > 1:
                        firstSlot = True
                        for i in range(sentence_part_len):
                            if firstSlot:
                                slots.append('b-' + entity_type)
                                firstSlot = False
                            else:
                                slots.append('i-' + entity_type)
                    else:
                        slots.append('b-' + entity_type)
                else:
                    for i in range(sentence_part_len):
                        slots.append('o')
            formatted_lines.append(construct_row(sentence, intent_category, slots))
    print(f'Finished converting file at path {json_file_path}. Writing to file...')
    write_to_file(intent_category, formatted_lines)
    

def construct_row(sentence, intent, slots):
    row = ''
    row += sentence
    row += '#!#'
    row += intent
    row += '#!#'
    row += ' '.join(slots)
    row += '\n'
    return row


def write_to_file(intent, lines):
    if intent is None or intent == '':
        print('No intent')
        return
        
    base_output_directory = 'data/corpus/'
    output_file_path = base_output_directory + intent + '/' + str(uuid.uuid4()) + '.txt'

    if not os.path.exists(base_output_directory + intent): 
        os.makedirs(base_output_directory + intent)
    
    with open(output_file_path, 'a') as output_file:
        output_file.writelines(lines)


def iterate_over_json_files_in_directory(directory_path):
    if not os.path.exists(directory_path) or directory_path is not os.path.isdir(directory_path):
        print(f"Das Pfad {directory_path} existiert nicht oder ist kein Verzeichnis.")
        
    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):  # Nur JSON-Dateien berücksichtigen
            file_path = os.path.join(directory_path, filename)
            convert_file(file_path)
    print('Finished converting all files!')
    
clean_formatted('data/corpus')
iterate_over_json_files_in_directory('data')

Pfad data/corpus ist kein Verzeichnis oder existiert nicht
Das Pfad data existiert nicht oder ist kein Verzeichnis.
Try converting file at path data/validate_PlayMusic.json
Finished converting file at path data/validate_PlayMusic.json. Writing to file...
Try converting file at path data/train_SearchCreativeWork_full.json
Finished converting file at path data/train_SearchCreativeWork_full.json. Writing to file...
Try converting file at path data/train_AddToPlaylist_full.json
Finished converting file at path data/train_AddToPlaylist_full.json. Writing to file...
Try converting file at path data/train_RateBook_full.json
Finished converting file at path data/train_RateBook_full.json. Writing to file...
Try converting file at path data/train_SearchScreeningEvent_full.json
Finished converting file at path data/train_SearchScreeningEvent_full.json. Writing to file...
Try converting file at path data/validate_GetWeather.json
Finished converting file at path data/validate_GetWeather.json. Writi

Als nächstes wird geprüft, ob auch alle Einträge in der Textdatei enthalten sind. Dafür werden die Einträge in der train- und validate.json mit der Anzahl der Zeilen in der Textdatei verglichen.

In [18]:
!wc -l data/formatted/RateBook.txt

2056 data/formatted/RateBook.txt


In [19]:
!jq '.RateBook | length' data/train_RateBook_full.json

1956


In [20]:
!jq '.RateBook | length' data/validate_RateBook.json

100


Man sieht, dass die Anzahl der `data`-Blöcke aus den JSON-Dateien der Anzahl der Zeilen in der erzeugten Textdatei entspricht. Die Konvertierung war also erfolgreich.

In [21]:
!head -n 10 data/formatted/RateBook.txt

BOS rate The Lotus and the Storm zero of 6 EOS RateBook o b-object_name i-object_name i-object_name i-object_name i-object_name b-rating_value o b-best_rating
BOS Rate The Fall-Down Artist 5 stars. EOS RateBook o b-object_name i-object_name i-object_name b-rating_value b-rating_unit o
BOS Rate the current novel one points EOS RateBook o o b-object_select b-object_type b-rating_value b-rating_unit
BOS rate The Ape-Man Within 4 EOS RateBook o b-object_name i-object_name i-object_name b-rating_value
BOS I give The Penalty three stars EOS RateBook o o b-object_name i-object_name b-rating_value b-rating_unit
BOS rate this novel a 4 EOS RateBook o b-object_select b-object_type o b-rating_value
BOS give 5 out of 6 points to Absolutely, Positively Not series EOS RateBook o b-rating_value o o b-best_rating b-rating_unit o b-object_name i-object_name i-object_name b-object_part_of_series_type
BOS I give Emile, or On Education five points. EOS RateBook o o b-object_name i-object_name i-object_nam

Quellen

* 1: https://investors.sonos.com/news-and-events/investor-news/latest-news/2019/Sonos-Announces-Acquisition-of-Snips/default.aspx, [Online, 07.03.2025]
* 2: Coucke A. et al., "Snips Voice Platform: an embedded Spoken Language Understanding system for private-by-design voice interfaces." 2018, [Online: https://arxiv.org/abs/1805.10190, 07.03.2025]
* 3: NLTK Team, tree2conlltags, [Online: https://www.nltk.org/_modules/nltk/chunk/util.html#tree2conlltags, 13.03.2025]
* 4: Explosion, spaCy convert, [Online: https://spacy.io/api/cli#converters, 13.03.2025]
* 5: Ramshaw und Marcus, "Text Chunking using Transformation-Based Learning" 1995, [Online: https://arxiv.org/abs/cmp-lg/9505040, 14.03.2025]

## Erstellen eines Corpus

Nachdem nun die Daten in ein verwendbares Format tranformiert wurden, müssen wir den effiziente Zugriff auf die Daten sicherstellen. Dafür bietet die Python-Bibliothek NLTK verschiedene `CorpusReader` Klassen zur Verfügung. Diese Klassen ermöglichen über Methoden den Zugriff auf die Dokumente selbst, sowie auf weitere Dateien eines Corpus, wie zum Beispiel die Lizenz des Corpus oder eine README.md Datei. NLTK bietet eine große Anzahl an CorpusReadern für verschiedenste Szenarien an. Ein einfacher CorpusReader ist der `PlainTextCorpusReader` [7]. Dieser bietet Zugriff auf reine Textdateien. Eine vollständige Liste ist in der [Dokumentation](https://www.nltk.org/api/nltk.corpus.reader.html) von NLTK zu finden.

Für das zuvor definierte Format gibt es keinen passenden vorgefertigten Reader. Daher müssen wir einen eigenen erstellen. Dafür kann die `CorpusReader`-Basisklasse [8] erweitert werden oder ein bestehender verwendet werden. Wir werden den `CategorizedPlaintextCorpusReader` als Basis verwenden. Dieser ist eine Erweiterung des zuvor erwähnten `PlainTextCorpusReader`, welcher Zugriff auf Textdateien bietet.  Durch die Nutzung des `CategorizedPlaintextCorpusReader` können die Dateien zusätzlich in Kategorien unterteilt werden. Dies geschieht über reguläre Ausdrücke. Im nachfolgenden Code definiert der reguläre Ausdruck _category_pattern_ die Kategorie als den Namen des Verzeichnisses, indem die Textdateien liegen.

In [5]:
from nltk.corpus.reader import CategorizedPlaintextCorpusReader
from nltk import wordpunct_tokenize
import codecs

In [6]:
class IOBCorpusReader(CategorizedPlaintextCorpusReader):
    
    def __init__(self, root, fileids, cat_pattern, encoding='utf-8'):
        super().__init__(root, fileids, cat_pattern=cat_pattern, encoding=encoding)

    def resolve(self, fileids=None, categories=None):

        if fileids is not None and categories is not None:
            raise ValueError('Specify only one')

        if categories is not None:
            return self.fileids(categories)
        else:
            return fileids

    def docs(self, fileids=None, categories=None):

        fileids = self.resolve(fileids, categories)

        for path, encoding in self.abspaths(fileids, include_encoding=True):
            with codecs.open(path, 'r', encoding=encoding) as file:
                yield file.read()
        
    def lines(self, fileids=None, categories=None):

        for doc in self.docs(fileids, categories):
            for line in doc.split('\n'):
                yield line

    def line_parts(self, fileids=None, categories=None):

        for line in self.lines(fileids, categories):
            if line == '':
                continue
            yield line.split('#!#')

    def intents(self, fileids=None, categories=None):

        for sentence, intent, labels in self.line_parts(fileids, categories):
            yield (sentence, intent)

    def slots(self, fileids=None, categories=None):

        for sentence, intent, slots in self.line_parts(fileids, categories):
            yield (sentence, slots.split())

    def joint_data(self, fileids=None, categories=None):

        for (sentence, intent), (sentence, slots) in zip(self.intents(fileids, categories), self.slots(fileids, categories)):
            yield (sentence, intent, slots)


    
file_pattern = r'.*\.txt'
category_pattern = r'([^/]+)/[^/]+\.txt$'
corpus = IOBCorpusReader('data/corpus', file_pattern, cat_pattern=category_pattern)
        

Quellen

* 6: Bengfort, Benjamin, et al. Applied Text Analysis with Python : Enabling Language-Aware Data Products with Machine Learning, O'Reilly Media, Incorporated, 2018. ProQuest Ebook Central, https://ebookcentral.proquest.com/lib/fh-swf/detail.action?docID=5425029.
* 7: NLTK Project, PlainTextCorpusReader, 2024, [Online: https://www.nltk.org/api/nltk.corpus.reader.plaintext.html#nltk.corpus.reader.plaintext.PlaintextCorpusReader, 17.03.2025]
* 8: NLTK project, CorpusReader, 2024, [Online: https://www.nltk.org/api/nltk.corpus.reader#nltk.corpus.reader.CorpusReader, 17.03.2025]

## Modelle

Beschreibung von BERT !!

In [13]:
from collections import defaultdict
import torch

In [14]:
intent_label_to_id = defaultdict(lambda: len(intent_label_to_id))
slot_label_to_id = defaultdict(lambda: len(slot_label_to_id))


In [16]:
from transformers import BertTokenizer

# Tokenizer von Hugging Face laden 
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

# Daten aus Corpus laden
joint_dataset = list(corpus.joint_data(categories='AddToPlaylist'))

input_ids_list = []
attention_masks_list = []
intent_labels_list = []
slot_labels_list = []

for data in joint_dataset:
    # Tokenisierung des Satzes mit dme BERT-Tokenizer
    encoding = tokenizer(data[0], padding='max_length', truncation=True, max_length=128, return_tensors="pt", is_split_into_words=True)

    # IOB-Tags an Subtoken-Länge anpassen
    iob_tags = data[2]
    aligned_slot_labels = []
    for word, label in zip(data[0], iob_tags):
        sub_tokens = tokenizer.tokenize(word)
        aligned_slot_labels.extend([label] * len(sub_tokens))
    
    # Abschneiden oder Auffüllen der Slot-Labels
    aligned_slot_labels = aligned_slot_labels[:128] + ["o"] * (128 - len(aligned_slot_labels))

    # Konvertieren in numerische Repräsentationen
    slot_label_ids = [slot_label_to_id[label] for label in aligned_slot_labels]
    intent_label_id = intent_label_to_id[data[1]]

    # Tensoren zur Liste hinzufügen
    input_ids_list.append(encoding["input_ids"].squeeze(0))  # Entfernt die Batch-Dimension
    attention_masks_list.append(encoding["attention_mask"].squeeze(0))  # Entfernt die Batch-Dimension
    slot_labels_list.append(torch.tensor(slot_label_ids, dtype=torch.long))  # Konvertieren in Tensor
    intent_labels_list.append(intent_label_id)

# Listen in einzelne Tensoren umwandeln
input_ids = torch.stack(input_ids_list)  # [batch_size, seq_length]
attention_masks = torch.stack(attention_masks_list)  # [batch_size, seq_length]
slot_labels = torch.stack(slot_labels_list)  # [batch_size, seq_length]
intent_labels = torch.tensor(intent_labels_list, dtype=torch.long)  # [batch_size]


In [ ]:
from transformers import BertForSequenceClassification

# Intent Detection Modell
intent_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=len(intent_label_to_id))

# Training vorbereiten
outputs = intent_model(input_ids=torch.tensor(input_ids), attention_mask=torch.tensor(attention_masks), labels=torch.tensor(intent_labels))
loss = outputs.loss
logits = outputs.logits


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipykernel_103/1346844624.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  outputs = intent_model(input_ids=torch.tensor(input_ids), attention_mask=torch.tensor(attention_masks), labels=torch.tensor(intent_labels))


In [12]:
from transformers import BertForTokenClassification

# Slot Filling Modell
slot_model = BertForTokenClassification.from_pretrained("bert-base-uncased", num_labels=len(slot_label_to_id))

# Training vorbereiten
outputs = slot_model(input_ids=torch.cat(input_ids, dim=0), attention_mask=torch.cat(attention_masks, dim=0), labels=torch.tensor(slot_labels))
loss = outputs.loss
logits = outputs.logits


Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
from transformers import BertModel
import torch.nn as nn

class JointIntentSlotModel(nn.Module):
    def __init__(self, num_intents, num_slots):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")
        self.intent_classifier = nn.Linear(self.bert.config.hidden_size, num_intents)
        self.slot_classifier = nn.Linear(self.bert.config.hidden_size, num_slots)
    
    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        sequence_output = outputs.last_hidden_state  # Für Slots
        pooled_output = outputs.pooler_output       # Für Intents

        # Intents und Slots vorhersagen
        intent_logits = self.intent_classifier(pooled_output)
        slot_logits = self.slot_classifier(sequence_output)
        
        return intent_logits, slot_logits


In [ ]:
# Verlustfunktionen
intent_loss_fn = nn.CrossEntropyLoss()
slot_loss_fn = nn.CrossEntropyLoss()

# Modell-Output
model = JointIntentSlotModel(num_intents=len(intent_label_to_id), num_slots=len(slot_label_to_id))
intent_logits, slot_logits = model(torch.tensor(input_ids), torch.tensor(attention_masks))

# Berechnung des Verlusts
intent_loss = intent_loss_fn(intent_logits, torch.tensor(intent_labels))
slot_loss = slot_loss_fn(slot_logits.view(-1, len(slot_label_to_id)), torch.tensor(slot_labels).view(-1))

# Gesamtverlust
loss = intent_loss + slot_loss


In [ ]:
from sklearn.metrics import classification_report

# Intent-Evaluierung
intent_preds = torch.argmax(intent_logits, dim=1).numpy()
print(classification_report(intent_labels, intent_preds, target_names=intent_label_to_id.keys()))

# Slot-Evaluierung
slot_preds = torch.argmax(slot_logits, dim=2).numpy()
for true, pred in zip(slot_labels, slot_preds):
    print("True:", [id_to_slot_label[i] for i in true])
    print("Pred:", [id_to_slot_label[i] for i in pred])


## Zusammenfassung